In [4]:
!pip install langgraph langchain langchain_openai


  Using cached langgraph-0.6.7-py3-none-any.whl.metadata (6.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 12.4 MB/s eta 0:00:00


In [58]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict

In [59]:
# define state class
class BMIState(TypedDict):
    bmi_score : float
    height_m : float
    weight_kg : float
    category : str
    health_advice: str

In [ ]:
# define python functions

# calculate bmi function
def calculate_bmi(state: BMIState) -> BMIState:
  height = state['height_m']
  weight = state['weight_kg']
  bmi = weight / (height ** 2)
  state['bmi_score'] = round(bmi, 2)
  return state

# define category label of the bmi
def category_label(state: BMIState) -> BMIState:
  bmi = state['bmi_score']
  if bmi < 18.5:
      state["category"] = "Underweight"
  elif 18.5 <= bmi < 25:
      state["category"] = "Normal"
  elif 25 <= bmi < 30:
      state["category"] = "Overweight"
  else:
      state["category"] = "Overweight(High Risk)"

  return state

# define a llm response
def llm_response(state: BMIState) -> BMIState:
  bmi_score = state['bmi_score']
  category = state['category']
  llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo",
                   api_key="")

  prompt = f'Here is BMI score: {bmi_score} and Here is category label based on the score {category}. Now Give user a best advise so that he can achive or maintain perfect BMI score'
  result = llm.invoke(prompt)
  state['health_advice'] = result.content
  return state

In [64]:
# define graph
graph = StateGraph(BMIState)
# add nodes to the graph
graph.add_node('calculate_bmi', calculate_bmi)
graph.add_node('label_bmi', category_label)
graph.add_node('llm_response', llm_response)

# add edges to the graph
graph.add_edge(START, 'calculate_bmi')
graph.add_edge('calculate_bmi', 'label_bmi')
graph.add_edge('label_bmi', 'llm_response')
graph.add_edge('llm_response', END)

# compile graph
workflow = graph.compile()

In [65]:
# execute the graph
# define initial state with inputs
initial_state = {"height_m":2, "weight_kg":70}
final_state = workflow.invoke(initial_state)
print(final_state)

{'bmi_score': 17.5, 'height_m': 2, 'weight_kg': 70, 'category': 'Underweight', 'health_advice': "To achieve or maintain a healthy BMI score, it is important to focus on a balanced diet and regular exercise. Make sure to include a variety of fruits, vegetables, whole grains, lean proteins, and healthy fats in your meals. Avoid crash diets or extreme restrictions, as they can be harmful to your overall health.\n\nIncorporate regular physical activity into your routine, aiming for at least 150 minutes of moderate exercise per week. This can include activities such as walking, jogging, cycling, or strength training. Remember to listen to your body and give yourself rest days as needed.\n\nConsult with a healthcare professional or a registered dietitian for personalized advice and guidance on achieving and maintaining a healthy BMI score. They can help you create a sustainable plan that works for your individual needs and lifestyle. Remember, it's important to focus on overall health and we